## Final Attempt at Optimization
In this notebook, I do the following:
* Drop 2 addtional features (`SPECIAL_CONSIDERATIONS` and `STATUS`)
* Group `APPLICATION_TYPE` less than 1000 in total count to "Other"
* Group `CLASSIFICATION` for those with less than 1000 in total count to "Other
* Utilize `keras-tuner` to determine the optimal number of layers and neurons in each later.

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
## In this test, remove 'Special_Conisderations' + 'Status' features.
application_df = application_df.drop(columns=['EIN','NAME','SPECIAL_CONSIDERATIONS', 'STATUS'])

In [7]:
# Determine the number of unique values in each column.
application_df.nunique()

,0
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
INCOME_AMT,9
ASK_AMT,8747
IS_SUCCESSFUL,2


In [8]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
app_type_counts

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [9]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = app_type_counts[app_type_counts < 1000].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
Other,2266
T4,1542
T6,1216
T5,1173
T19,1065


In [10]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
...,...
C4120,1
C8210,1
C2561,1


In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = class_counts[class_counts < 1000].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
Other,2261
C3000,1918
C2100,1883


In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric_app_data_df = pd.get_dummies(application_df)
numeric_app_data_df

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,True,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,108590,1,False,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,5000,0,False,False,False,False,True,False,True,False,...,False,True,False,False,False,False,False,False,False,False
3,6692,1,False,False,True,False,False,False,True,False,...,True,False,False,True,False,False,False,False,False,False
4,142590,1,False,False,True,False,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,False,False,False,True,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
34295,5000,0,False,False,False,True,False,False,True,False,...,False,True,False,False,False,False,False,False,False,False
34296,5000,0,False,False,True,False,False,False,True,False,...,False,True,False,False,False,False,False,False,False,False
34297,5000,1,False,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [13]:
# Split our preprocessed data into our features and target arrays

# Labels
y = numeric_app_data_df['IS_SUCCESSFUL']

# Features
X = numeric_app_data_df.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Define the model and use `keras-tuner` to optimize hyperparameters

In [24]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):

  inputdim = len(X_train_scaled[0])

  # create basic model
  nn_model = tf.keras.models.Sequential()

  # Allow kerastuner to decide which activation funciton to use in hidden layers
  activation = hp.Choice('activation',['relu','tanh'])

  # Allow kerastuner to decide number of neurons in first layer.
  nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=32,
        max_value=512,
        step=32), activation=activation, input_dim= inputdim))

  # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
  for i in range(hp.Int('num_layers', 1, 6)):
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=16,
            max_value=128,
            step=16),
            activation=activation))

  # Tune the learning rate for the optimizer; choose an optimal value form 0.01, 0.001, 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values= [1e-2, 1e-3, 1e-4])

  # Output layer
  nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

  # Compile the model
  nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

  return nn_model

In [2]:
# !pip install keras-tuner
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.6 MB/s eta 0:00:00


In [5]:
# Import the kerastuner library
import keras_tuner as kt

In [25]:
# Initiate the tuner
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [26]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train, epochs=20, validation_data= (X_test_scaled, y_test))

Trial 60 Complete [00h 01m 06s]
val_accuracy: 0.731195330619812

Best val_accuracy So Far: 0.7316617965698242
Total elapsed time: 00h 32m 04s


In [27]:
# Get best model hyperparameters
best_hp = tuner.get_best_hyperparameters(1)[0].values
best_hp

{'activation': 'relu',
 'first_units': 512,
 'num_layers': 3,
 'units_0': 32,
 'learning_rate': 0.0001,
 'units_1': 64,
 'units_2': 16,
 'units_3': 112,
 'units_4': 96,
 'units_5': 48,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0043'}

In [30]:
# Check structure of "best" kt optimized model
kt_optimized_model = tuner.get_best_models(1)[0]
kt_optimized_model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 512)                 │          19,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │          16,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 16)                  │           1,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 39,041 (152.50 KB)

 Trainable params: 39,041 (152.50 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
# Evaluate best model against full test data
model_loss, model_accuracy = kt_optimized_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 3ms/step - accuracy: 0.7317 - loss: 0.5532
Loss: 0.5531882643699646, Accuracy: 0.7316617965698242


In [41]:
# Export Best Model
kt_optimized_model.save("Models/AlphabetSoupCharity_Optimization_v2.h5")

## Optimization 3 Results
Through the use of the `keras-tuner` optimized model, I achieved an accuracy rating of 73% and a loss of 55%. This is similar to the accuracy achieved in my first Optimization attempt in `AlphabetSoupCharity_Model_Optimization_v1.ipynb`, though the `kt_optimized_model` has a lower loss rating. Still unable to achieve the goal accuracy percentage of 75% after 3 attempts.